In [2]:
## Loading of libraries

import numpy as np
import ProjectionPlane
import MacroVision
import MicroVision
import rtde_control
import time
import robotiq_gripper 
import RPYtoRV
import main_triangle_class as Slider
import rtde_receive
import RotationAngle


In [3]:
## Constant variables

MaxSpeed= 0.4                               # m/s
MaxAcceleration=0.2                         # m/s2
StdSpeed=0.15                               # m/s
StdAcceleration=0.15                        # m/s2
SlowSpeed=0.01                              # m/s
SlowAcceleration=0.01                       # m/s2
GripperMaxSpeed=255                         # 0-255 Grading
GripperSlowSpeed=150                        # 0-255 Grading
GripperMaxForce=255                         # 0-255 Grading
GripperSlowForce=125                        # 0-255 Grading
SleepTime=0.1


In [4]:
## Initialisation of variables

theta_rad = 0.0                                                     #Taskboard Orientation angle
Home_Pose = [-0.055,-0.500,0.550,0,3.14,0]                          #Robot Initial Home pose
MacroWorldCoordinates = [0.0,0.0]
RedButtonWorldCoords = [0.0,0.0]
Task1_Pose = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Image_capturing_Pose = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]               #Image Capturing pose for slider task           


In [5]:
## Initial Connection and Calibration

ip = "192.168.31.99"                                #IP address of robot
rtde_c = rtde_control.RTDEControlInterface(ip)
rtde_r = rtde_receive.RTDEReceiveInterface(ip)

if rtde_c.isConnected() == False:
    print("Robot is disconnected. Trying to reconnect...")
    rtde_c.reconnect()

rtde_c.moveL([-0.055,-0.500,0.550,0,3.14,0],MaxSpeed,MaxAcceleration)  #Moving to Home position
time.sleep(SleepTime)

## Gripper initialisation

print("Creating gripper...")
gripper = robotiq_gripper.RobotiqGripper()
gripper.connect(ip, 63352)
print("Activating gripper...")
gripper.activate()
gripper.move_and_wait_for_pos(255, 255, 255)

Creating gripper...
Activating gripper...
Gripper auto-calibrated to [3, 246]


(246, <ObjectStatus.AT_DEST: 3>)

In [6]:
## Blue Button Pressing

def Task1():

    print("Task1 initiated..")

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()

    gripper.move_and_wait_for_pos(255,GripperMaxSpeed,GripperMaxForce)

    tool_rpy = (3.14,0.0,3.14+theta_rad)                            
    tool_rv = RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2]) # Rotational vector values for rotating tool to taskboard orientation

    BlueButtonCentre = (0,0.01425+0.003)                          # Co ordinates of Blue button wrt Red button
    BlueButtonCentre_Calculated = ((BlueButtonCentre[0]*np.cos(theta_rad))-(BlueButtonCentre[1]*np.sin(theta_rad)),
                                    (BlueButtonCentre[1]*np.cos(theta_rad))+(BlueButtonCentre[0]*np.sin(theta_rad)))

    BlueButtonWorldCoords = (BlueButtonCentre_Calculated[0] + RedButtonWorldCoords[0], BlueButtonCentre_Calculated[1] + RedButtonWorldCoords[1])

    Task1_Pose[0] = BlueButtonWorldCoords[0]
    Task1_Pose[1] = BlueButtonWorldCoords[1]
    Task1_Pose[2] = 0.250
    Task1_Pose[3] = tool_rv[0]
    Task1_Pose[4] = tool_rv[1]
    Task1_Pose[5] = tool_rv[2]

    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    Task1_Pose[2] = 0.140
    rtde_c.moveL(Task1_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    Task1_Pose[2] = 0.130
    rtde_c.moveL(Task1_Pose,SlowSpeed,SlowAcceleration)
    time.sleep(SleepTime)

    Task1_Pose[2] = 0.250
    rtde_c.moveL(Task1_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)


    print(f"Task1 completed..")

In [7]:
## Probe replacement

def Task3():
    print("Task3 initiated..")

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()

    BlackHoleCentre = (-0.0566,0.012)                                             #Location of Black hole wrt Red button
    RedHoleCentre = (-0.0566,-0.013)                                              #Location of Red hole wrt Red button

    BlackHoleCentre_Calculated = ((BlackHoleCentre[0]*np.cos(theta_rad))-(BlackHoleCentre[1]*np.sin(theta_rad)),
                (BlackHoleCentre[1]*np.cos(theta_rad))+(BlackHoleCentre[0]*np.sin(theta_rad)))

    RedHoleCentre_Calculated = ((RedHoleCentre[0]*np.cos(theta_rad))-(RedHoleCentre[1]*np.sin(theta_rad)),
                (RedHoleCentre[1]*np.cos(theta_rad))+(RedHoleCentre[0]*np.sin(theta_rad)))

    BlackHoleWorldCoords = (BlackHoleCentre_Calculated[0] + RedButtonWorldCoords[0], BlackHoleCentre_Calculated[1] + RedButtonWorldCoords[1])

    RedHoleWorldCoords = (RedHoleCentre_Calculated[0] + RedButtonWorldCoords[0],RedHoleCentre_Calculated[1] + RedButtonWorldCoords[1])

    print(f"Position of Black hole in Task board : {BlackHoleWorldCoords}")
    print(f"Position of Red hole in Task board : {RedHoleWorldCoords}")

    ## Picking up Probe from Black hole

    gripper.move_and_wait_for_pos(0, GripperMaxSpeed, GripperMaxForce)

    Task2_Pose = Task1_Pose
    Task2_Pose[0] = BlackHoleWorldCoords[0]
    Task2_Pose[1] = BlackHoleWorldCoords[1]
    Task2_Pose[2] = 0.200

    rtde_c.moveL(Task2_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    Task2_Pose[2] = 0.133
    rtde_c.moveL(Task2_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(230, GripperSlowSpeed, GripperSlowForce)

    Task2_Pose[2] = 0.200
    rtde_c.moveL(Task2_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ## Placing Probe in Red hole

    Task2_Pose[0] = RedHoleWorldCoords[0]
    Task2_Pose[1] = RedHoleWorldCoords[1]
    rtde_c.moveL(Task2_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    Task2_Pose[2] = 0.133
    rtde_c.moveL(Task2_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)
   

    gripper.move_and_wait_for_pos(50, GripperMaxSpeed, GripperMaxForce)
    Task2_Pose[2] = 0.200
    rtde_c.moveL(Task2_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    print("Task2 completed..")

In [8]:
## Yellow slider moving


def Task2a():

    print("Task 2 initiated..")  

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()


    ## Definition of Image capturing position  
    # M5buttonPos=(0.0,(0.13776-0.070))         #To position the camera above M5 , Distance from Red button to M5 is subtracted from camera offset    

    # M5buttonPos_Calculated=((M5buttonPos[0]*np.cos(theta_rad))-(M5buttonPos[1]*np.sin(theta_rad)),
    #     (M5buttonPos[1]*np.cos(theta_rad))+(M5buttonPos[0]*np.sin(theta_rad)))
    
    # M5button_World=(M5buttonPos_Calculated[0]+RedButtonWorldCoords[0],M5buttonPos_Calculated[1]+RedButtonWorldCoords[1]) 

    # print(M5button_World)
     
    # Image_capturing_Pose=Task1_Pose
    # Image_capturing_Pose[0]=M5button_World[0]
    # Image_capturing_Pose[1]=M5button_World[1]
    # Image_capturing_Pose[2]=0.250

    # print(Image_capturing_Pose)

    ## Moving to Image capturing position to capture yellow triangle location
    # rtde_c.moveL(Image_capturing_Pose,MaxSpeed,MaxAcceleration)
    # time.sleep(SleepTime)
    # time.sleep(4)

    # image1 = Slider.CaptureSlider()

    # RedToYellow=float((Slider.R2Y_dist(image1))/1000)

    # if RedToYellow==0.0: 
    #     print("Image not detected.. Returning to Main program..") 
    #     return None
    
    # # print(RedToYellow)

    # ## Definition of Slider starting position

    gripper.move_and_wait_for_pos(140,GripperMaxSpeed,GripperMaxForce)
    SliderStartPos = [-0.0355,-0.078]
    SliderStartPos_Calculated = ((SliderStartPos[0]*np.cos(theta_rad))-(SliderStartPos[1]*np.sin(theta_rad)),
        (SliderStartPos[1]*np.cos(theta_rad))+(SliderStartPos[0]*np.sin(theta_rad)))

    SliderStartWorld = (SliderStartPos_Calculated[0]+RedButtonWorldCoords[0],SliderStartPos_Calculated[1]+RedButtonWorldCoords[1])
    
    Task3_Pose1 = Task1_Pose

    Task3_Pose1[0]=SliderStartWorld[0]
    Task3_Pose1[1]=SliderStartWorld[1]

    
    rtde_c.moveL(Task3_Pose1,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)
    Task3_Pose1[2]=0.133
    
    rtde_c.moveL(Task3_Pose1,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    
    gripper.move_and_wait_for_pos(230, GripperSlowSpeed, GripperSlowForce)      #90% gripper
    
    ## Definition of Yellow starting position

    Yellow_Pos=SliderStartPos
    Yellow_Pos[1]=Yellow_Pos[1]+0.0155
    

    Yellow_Pos_Calculated=((Yellow_Pos[0]*np.cos(theta_rad))-(Yellow_Pos[1]*np.sin(theta_rad)),
        (Yellow_Pos[1]*np.cos(theta_rad))+(Yellow_Pos[0]*np.sin(theta_rad)))

    Yellow_Pos_World=(Yellow_Pos_Calculated[0]+RedButtonWorldCoords[0],Yellow_Pos_Calculated[1]+RedButtonWorldCoords[1])
    

    Task3_Pose2= Task1_Pose

    Task3_Pose2[0]=Yellow_Pos_World[0]
    Task3_Pose2[1]=Yellow_Pos_World[1]
    Task3_Pose2[2]=0.133

    ## Moving the slider to Yellow triangle
    

    rtde_c.moveL(Task3_Pose2,SlowSpeed,SlowAcceleration)
    time.sleep(SleepTime)
    gripper.move_and_wait_for_pos(140, GripperMaxSpeed, GripperMaxForce)
    Task3_Pose2[2]=0.250
    rtde_c.moveL(Task3_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    print("Task 2a completed..")


    

In [9]:
## Green Slider moving


def Task2B():

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()

    print("Task 2b started..")

    # if Yellow_Pose==None:
    #     print("Image not detected.. Returning to Main program..") 
    #     return None 


    ## Definition of Image capturing position  
    M5buttonPos=(0.0,(0.13776-0.070))         #To position the camera above M5 , Distance from Red button to M5 is subtracted from camera offset    

    M5buttonPos_Calculated=((M5buttonPos[0]*np.cos(theta_rad))-(M5buttonPos[1]*np.sin(theta_rad)),
        (M5buttonPos[1]*np.cos(theta_rad))+(M5buttonPos[0]*np.sin(theta_rad)))

    M5button_World=(M5buttonPos_Calculated[0]+RedButtonWorldCoords[0],M5buttonPos_Calculated[1]+RedButtonWorldCoords[1]) 

        
    Image_capturing_Pose=Task1_Pose
    Image_capturing_Pose[0]=M5button_World[0]
    Image_capturing_Pose[1]=M5button_World[1]
    Image_capturing_Pose[2]=0.250


    ## Moving to Image capturing position to capture yellow triangle location

    rtde_c.moveL(Image_capturing_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    image2 = Slider.CaptureSlider()
    RedToGreen= float(Slider.R2G_dist(image2)/1000)

    if RedToGreen==0.0:
        print("Image not detected.. Returning to Main program..") 
        return None
    
    if RedToGreen<0:
        RedToGreen-=0.001
    
    if RedToGreen>0:
        RedToGreen+=0.001
     

    Yellow_Pose=[-0.0355,-0.078+0.0155]

    Current_Pos=Yellow_Pose

    Current_Pos_Calculated = ((Current_Pos[0]*np.cos(theta_rad))-(Current_Pos[1]*np.sin(theta_rad)),
        (Current_Pos[1]*np.cos(theta_rad))+(Current_Pos[0]*np.sin(theta_rad)))

    CurrentPosWorld = (Current_Pos_Calculated[0]+RedButtonWorldCoords[0],Current_Pos_Calculated[1]+RedButtonWorldCoords[1])


    gripper.move_and_wait_for_pos(140,GripperMaxSpeed,GripperMaxForce)

    Task3b_Pose1 = Task1_Pose    
    Task3b_Pose1[0]=CurrentPosWorld[0]
    Task3b_Pose1[1]=CurrentPosWorld[1]

    ## Moving the slider to existing position from previous task
    
    rtde_c.moveL(Task3b_Pose1,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)
    Task3b_Pose1[2]=0.133
    rtde_c.moveL(Task3b_Pose1,StdSpeed,StdAcceleration)
    gripper.move_and_wait_for_pos(230, GripperSlowSpeed, GripperSlowForce)   

    ## Calculation of Green triangle position

    GreenPos=Yellow_Pose
    # GreenPos[1]=GreenPos[1]-(RedToGreen+0.0015)
    GreenPos[1]=GreenPos[1]-RedToGreen

    Green_Pos_Calculated=((GreenPos[0]*np.cos(theta_rad))-(GreenPos[1]*np.sin(theta_rad)),
        (GreenPos[1]*np.cos(theta_rad))+(GreenPos[0]*np.sin(theta_rad)))

    Green_Pos_World=(Green_Pos_Calculated[0]+RedButtonWorldCoords[0],Green_Pos_Calculated[1]+RedButtonWorldCoords[1])

    Task3b_Pose2= Task1_Pose

    Task3b_Pose2[0]=Green_Pos_World[0]
    Task3b_Pose2[1]=Green_Pos_World[1]
   
    Task3b_Pose1[2]=0.133

    rtde_c.moveL(Task3b_Pose2,SlowSpeed,SlowAcceleration)
    time.sleep(SleepTime)
    gripper.move_and_wait_for_pos(140, GripperMaxSpeed, GripperMaxForce)
    Task3b_Pose2[2]=0.250
    rtde_c.moveL(Task3b_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    print("Task 2b completed..")


In [10]:
## Door Opening

def Task4():

    print("Task 4 initiated..")

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()

    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)
    
    gripper.move_and_wait_for_pos(255,GripperMaxSpeed,GripperMaxForce)
    
    KnobStartPos = (-0.122,-0.006+0.009)

    KnobStartPos_Calculated = ((KnobStartPos[0]*np.cos(theta_rad))-(KnobStartPos[1]*np.sin(theta_rad)),
                                    (KnobStartPos[1]*np.cos(theta_rad))+(KnobStartPos[0]*np.sin(theta_rad)))
    
    KnobStartPos_world = (KnobStartPos_Calculated[0] + RedButtonWorldCoords[0], KnobStartPos_Calculated[1] + RedButtonWorldCoords[1])

    Task4_Pose = Task1_Pose
    Task4_Pose[0] = KnobStartPos_world[0]
    Task4_Pose[1] = KnobStartPos_world[1]
    Task4_Pose[2] = 0.132 

    rtde_c.moveL(Task4_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    KnobEndPos = (-0.122,-0.0674+0.003)

    KnobEndPos_Calculated = ((KnobEndPos[0]*np.cos(theta_rad))-(KnobEndPos[1]*np.sin(theta_rad)),
                                    (KnobEndPos[1]*np.cos(theta_rad))+(KnobEndPos[0]*np.sin(theta_rad)))
    
    KnobEndPos_world = (KnobEndPos_Calculated[0] + RedButtonWorldCoords[0], KnobEndPos_Calculated[1] + RedButtonWorldCoords[1])
    
    ## Computation of Local points for Door movement

    localpoints = [[-0.122 , -4.4450422655897825e-05 , 0.14752914270615125],
                 [-0.122 , -0.006038475772933677 , 0.162],
                 [-0.122 , -0.015573593128807142 , 0.17442640687119287],
                 [-0.122 , -0.02799999999999999 , 0.18296152422706633],
                 [-0.122 , -0.04247085729384875 , 0.18795554957734412]]
    
    waypoints = [[0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0]]
    
    for i in range(5):
        
        localpoints_Calculated = ((localpoints[i][0]*np.cos(theta_rad))-(localpoints[i][1]*np.sin(theta_rad)),
                                    (localpoints[i][1]*np.cos(theta_rad))+(localpoints[i][0]*np.sin(theta_rad)),
                                    (localpoints[i][2]))
    
        
        localpoints_world = (localpoints_Calculated[0] + RedButtonWorldCoords[0], localpoints_Calculated[1] + RedButtonWorldCoords[1], localpoints_Calculated[2])

        waypoints[i] = Task1_Pose
        waypoints[i][0] = localpoints_world[0]
        waypoints[i][1] = localpoints_world[1]
        waypoints[i][2] = localpoints_world[2]

        rtde_c.moveL(waypoints[i],StdSpeed,StdAcceleration)
        time.sleep(0.1)

    waypoints[i][2]=0.250
    rtde_c.moveL(waypoints[i],StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ## Door Pushing to ensure complete opening

    PushPos = (-0.122,0)

    PushPos_Calculated = ((PushPos[0]*np.cos(theta_rad))-(PushPos[1]*np.sin(theta_rad)),
                                    (PushPos[1]*np.cos(theta_rad))+(PushPos[0]*np.sin(theta_rad)))
    
    PushPos_world = (PushPos_Calculated[0] + RedButtonWorldCoords[0], PushPos_Calculated[1] + RedButtonWorldCoords[1])

    Task4_Pose[0]=PushPos_world[0]
    Task4_Pose[1]=PushPos_world[1]
    Task4_Pose[2]=0.25

    rtde_c.moveL(Task4_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    Task4_Pose[2]=0.180
    rtde_c.moveL(Task4_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    EndPos = (-0.122,-0.08)

    EndPos_Calculated = ((EndPos[0]*np.cos(theta_rad))-(EndPos[1]*np.sin(theta_rad)),
                                    (EndPos[1]*np.cos(theta_rad))+(EndPos[0]*np.sin(theta_rad)))
    
    EndPos_world = (EndPos_Calculated[0] + RedButtonWorldCoords[0], EndPos_Calculated[1] + RedButtonWorldCoords[1])

    Task4_Pose[0]=EndPos_world[0]
    Task4_Pose[1]=EndPos_world[1]

    rtde_c.moveL(Task4_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    Task4_Pose[2]=0.25

    rtde_c.moveL(Task4_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)


    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    print("Task 4a completed..")
    
    

In [11]:
## Probing

def Task4b():

    print("Task 4b initiated..")

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()

    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(255,GripperMaxSpeed,GripperMaxForce)

    ## Picking up of Probe    

    Probe_Pos=(-0.1976,0.020)
    Probe_Pos_Calculated = ((Probe_Pos[0]*np.cos(theta_rad))-(Probe_Pos[1]*np.sin(theta_rad)),
                                    (Probe_Pos[1]*np.cos(theta_rad))+(Probe_Pos[0]*np.sin(theta_rad)))
    
    Probe_Pos_world = (Probe_Pos_Calculated[0] + RedButtonWorldCoords[0], Probe_Pos_Calculated[1] + RedButtonWorldCoords[1])

    Task4_Pose = Task1_Pose
    Task4_Pose[0] = Probe_Pos_world[0]
    Task4_Pose[1] = Probe_Pos_world[1]
    Task4_Pose[2] = 0.250

    ## Rotating the tool 180 deg

    tool_rpy=[(3.14-0.7857),0.0,(3.14+theta_rad+3.14)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])
    Task4_Pose[3]=tool_rv[0]
    Task4_Pose[4]=tool_rv[1]
    Task4_Pose[5]=tool_rv[2]

    
    gripper.move_and_wait_for_pos(130, 255, 255)
    rtde_c.moveL(Task4_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    Task4_Pose[2]=0.135

    rtde_c.moveL(Task4_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)
    gripper.move_and_wait_for_pos(191, GripperSlowSpeed, GripperSlowForce) #75% gripper action


    Probe_Pos_End=(-0.1976,0.050)
    Probe_Pos_End_Calculated = ((Probe_Pos_End[0]*np.cos(theta_rad))-(Probe_Pos_End[1]*np.sin(theta_rad)),
                                    (Probe_Pos_End[1]*np.cos(theta_rad))+(Probe_Pos_End[0]*np.sin(theta_rad)))
    
    Probe_Pos_End_world = (Probe_Pos_End_Calculated[0] + RedButtonWorldCoords[0], Probe_Pos_End_Calculated[1] + RedButtonWorldCoords[1])

    Task4_Pose2=Task4_Pose
    Task4_Pose2[0]=Probe_Pos_End_world[0]
    Task4_Pose2[1]=Probe_Pos_End_world[1]

    rtde_c.moveL(Task4_Pose2,SlowSpeed,SlowAcceleration)
    time.sleep(SleepTime)
    Task4_Pose2[2]=0.300
    rtde_c.moveL(Task4_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)



    tool_rpy=[(3.14+0.7857),0.0,(3.14+theta_rad)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])
    Task4_Pose2[3]=tool_rv[0]
    Task4_Pose2[4]=tool_rv[1]
    Task4_Pose2[5]=tool_rv[2]

    rtde_c.moveL(Task4_Pose2,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    ## Moving to Probing point

    Probe_point=(-0.15+0.0006,-0.023-0.0077-0.003+0.0010-0.005)
    Probe_point_Calculated = ((Probe_point[0]*np.cos(theta_rad))-(Probe_point[1]*np.sin(theta_rad)),
                                (Probe_point[1]*np.cos(theta_rad))+(Probe_point[0]*np.sin(theta_rad)))
    
    Probe_Point_world = (Probe_point_Calculated[0] + RedButtonWorldCoords[0], Probe_point_Calculated[1] + RedButtonWorldCoords[1])


    Task4_Pose2[0]=Probe_Point_world[0]
    Task4_Pose2[1]=Probe_Point_world[1]
    # print(Task4_Pose2)    
    
    rtde_c.moveL(Task4_Pose2,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)   

    Task4_Pose2[2]=0.200
    rtde_c.moveL(Task4_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)
    Task4_Pose2[2]=0.189    
    rtde_c.moveL(Task4_Pose2,SlowSpeed,SlowAcceleration)
    time.sleep(SleepTime)
    Task4_Pose2[2]=0.3
    rtde_c.moveL(Task4_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ## Dropping the probe

    Task4_Pose2[0]=-0.34
    Task4_Pose2[1]=-0.45

    rtde_c.moveL(Task4_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    Task4_Pose2[2]=0.22

    rtde_c.moveL(Task4_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(50,GripperMaxSpeed,GripperMaxForce)

    tool_rpy=[(3.14),0.0,(3.14+theta_rad)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])
    Task4_Pose2[2]=0.3
    Task4_Pose2[3]=tool_rv[0]
    Task4_Pose2[4]=tool_rv[1]
    Task4_Pose2[5]=tool_rv[2]

    rtde_c.moveL(Task4_Pose2,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(255,GripperMaxSpeed,GripperMaxForce)

    print("Task 4 completed..")

    

In [12]:
## Wire Stowing

def Task5():

    print("Task 5 initiated...")

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect() 

    rtde_c.moveL(Task1_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    ## Picking up the wire    

    gripper.move_and_wait_for_pos(50, GripperSlowSpeed, GripperSlowForce)

    StartPoint=(-0.0576,0.0885+0.03-0.023)

    StartPoint_Calculated = ((StartPoint[0]*np.cos(theta_rad))-(StartPoint[1]*np.sin(theta_rad)),
                (StartPoint[1]*np.cos(theta_rad))+(StartPoint[0]*np.sin(theta_rad)))


    StartPointWorldCoords = (StartPoint_Calculated[0] + RedButtonWorldCoords[0], StartPoint_Calculated[1] + RedButtonWorldCoords[1])
    Task5_Pose=Task1_Pose

    Task5_Pose[0]=StartPointWorldCoords[0]
    Task5_Pose[1]=StartPointWorldCoords[1]
    Task5_Pose[2]=0.250

    tool_rpy=[(3.14+0.7857),0.0,(3.14+theta_rad)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])
    Task5_Pose[3]=tool_rv[0]
    Task5_Pose[4]=tool_rv[1]
    Task5_Pose[5]=tool_rv[2]

    rtde_c.moveL(Task5_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    Task5_Pose[2]=0.088
    rtde_c.moveL(Task5_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)   

    WirePickPoint=(-0.0576,0.0885+0.03-0.023-0.0246)

    WirePickPoint_Calculated = ((WirePickPoint[0]*np.cos(theta_rad))-(WirePickPoint[1]*np.sin(theta_rad)),
                (WirePickPoint[1]*np.cos(theta_rad))+(WirePickPoint[0]*np.sin(theta_rad)))


    WirePickPointWorldCoords = (WirePickPoint_Calculated[0] + RedButtonWorldCoords[0], WirePickPoint_Calculated[1] + RedButtonWorldCoords[1])

    Task5_Pose[0]=WirePickPointWorldCoords[0]
    Task5_Pose[1]=WirePickPointWorldCoords[1]

    rtde_c.moveL(Task5_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(255, GripperSlowSpeed, GripperSlowForce)


    time.sleep(SleepTime) 

    ## Defining local points for stowing path


    LocalPoints=([-0.210,0.0709,0.088],
             [-0.210,0.0709,0.163],
             [-0.210,0.0759,0.163],
             [0.039,0.0759,0.163],
             [0.039,0.0709,0.088])
    
    waypoints = [[0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0],
                 [0.0,0.0,0.0,0.0,0.0,0.0]]
    
    for i in range(2):

        for j in range(5):
        
            localpoints_Calculated = ((LocalPoints[j][0]*np.cos(theta_rad))-(LocalPoints[j][1]*np.sin(theta_rad)),
                                        (LocalPoints[j][1]*np.cos(theta_rad))+(LocalPoints[j][0]*np.sin(theta_rad)),
                                        (LocalPoints[j][2]))
            # print(localpoints_Calculated)
            
            localpoints_world = (localpoints_Calculated[0] + RedButtonWorldCoords[0], localpoints_Calculated[1] + RedButtonWorldCoords[1], localpoints_Calculated[2])

            waypoints[j] = Task5_Pose
            waypoints[j][0] = localpoints_world[0]
            waypoints[j][1] = localpoints_world[1]
            waypoints[j][2] = localpoints_world[2]

            rtde_c.moveL(waypoints[j],StdSpeed,StdAcceleration)
            time.sleep(0.1) 

    ## Wire release

    releasePoints=[-0.260,0.0809,0.088]

    releasePoints_Calculated = ((releasePoints[0]*np.cos(theta_rad))-(releasePoints[1]*np.sin(theta_rad)),
                                        (releasePoints[1]*np.cos(theta_rad))+(releasePoints[0]*np.sin(theta_rad)))
    
    
    releasepoints_world = (releasePoints_Calculated[0] + RedButtonWorldCoords[0], releasePoints_Calculated[1] + RedButtonWorldCoords[1], localpoints_Calculated[2])

    ReleasePose = Task5_Pose
    ReleasePose[0] = releasepoints_world[0]
    ReleasePose[1] = releasepoints_world[1]
    ReleasePose[2] = releasepoints_world[2]

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)
    ReleasePose[2]=0.16

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    tool_rpy=[(3.14),0.0,(3.14+theta_rad)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])

    ReleasePose[3]=tool_rv[0]
    ReleasePose[4]=tool_rv[1]
    ReleasePose[5]=tool_rv[2]

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    tool_rpy=[(3.14),0.0,(3.14+theta_rad+1.5714)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])

    ReleasePose[3]=tool_rv[0]
    ReleasePose[4]=tool_rv[1]
    ReleasePose[5]=tool_rv[2]

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)


    releasePoints=[-0.260,0.1009]

    releasePoints_Calculated = ((releasePoints[0]*np.cos(theta_rad))-(releasePoints[1]*np.sin(theta_rad)),
                                        (releasePoints[1]*np.cos(theta_rad))+(releasePoints[0]*np.sin(theta_rad)))
            
    
    releasepoints_world = (releasePoints_Calculated[0] + RedButtonWorldCoords[0], releasePoints_Calculated[1] + RedButtonWorldCoords[1])

    ReleasePose[0] = releasepoints_world[0]
    ReleasePose[1] = releasepoints_world[1]


    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ReleasePose[2]=0.046

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(125,GripperSlowSpeed,GripperSlowForce)
    

    releasePoints=[-0.370,0.1009]

    releasePoints_Calculated = ((releasePoints[0]*np.cos(theta_rad))-(releasePoints[1]*np.sin(theta_rad)),
                                        (releasePoints[1]*np.cos(theta_rad))+(releasePoints[0]*np.sin(theta_rad)))
            
    
    releasepoints_world = (releasePoints_Calculated[0] + RedButtonWorldCoords[0], releasePoints_Calculated[1] + RedButtonWorldCoords[1])

    ReleasePose[0] = releasepoints_world[0]
    ReleasePose[1] = releasepoints_world[1]

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(191,GripperSlowSpeed,GripperSlowForce)
    time.sleep(SleepTime)

    ReleasePose[2]=0.2

    rtde_c.moveL(ReleasePose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ## Probe pickup

    ProbeLocation=[-0.2016,0.01]

    ProbeLocationCalculated = ((ProbeLocation[0]*np.cos(theta_rad))-(ProbeLocation[1]*np.sin(theta_rad)),
                                        (ProbeLocation[1]*np.cos(theta_rad))+(ProbeLocation[0]*np.sin(theta_rad)))
            
    
    ProbeLocationworld = (ProbeLocationCalculated[0] + RedButtonWorldCoords[0], ProbeLocationCalculated[1] + RedButtonWorldCoords[1])

    ProbeInsertionPose=ReleasePose

    tool_rpy=[(3.14),0.0,(3.14+theta_rad+3.14)]
    tool_rv=RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2])

    ProbeInsertionPose[0]=ProbeLocationworld[0]
    ProbeInsertionPose[1]=ProbeLocationworld[1]       
    ProbeInsertionPose[2]=0.25
    ProbeInsertionPose[3]=tool_rv[0]
    ProbeInsertionPose[4]=tool_rv[1]
    ProbeInsertionPose[5]=tool_rv[2]

    rtde_c.moveL(ProbeInsertionPose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ProbeInsertionPose[2]=0.140

    rtde_c.moveL(ProbeInsertionPose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    ProbeLocation=[-0.2016,-0.02]

    ProbeLocationCalculated = ((ProbeLocation[0]*np.cos(theta_rad))-(ProbeLocation[1]*np.sin(theta_rad)),
                                        (ProbeLocation[1]*np.cos(theta_rad))+(ProbeLocation[0]*np.sin(theta_rad)))
            
    
    ProbeLocationworld = (ProbeLocationCalculated[0] + RedButtonWorldCoords[0], ProbeLocationCalculated[1] + RedButtonWorldCoords[1])

    ProbeInsertionPose[0]=ProbeLocationworld[0]
    ProbeInsertionPose[1]=ProbeLocationworld[1]

    rtde_c.moveL(ProbeInsertionPose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    gripper.move_and_wait_for_pos(125,GripperMaxSpeed,GripperMaxForce)

    ProbeInsertionPose[2]=0.25

    rtde_c.moveL(ProbeInsertionPose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    print("Task 5 completed..")    
    
        

In [13]:
## Red button pressing

def Task6():

    print("Task6 initiated..")

    if rtde_c.isConnected() == False:
        print("Robot is disconnected. Trying to reconnect...")
        rtde_c.reconnect()

    gripper.move_and_wait_for_pos(255,GripperMaxSpeed,GripperMaxForce)
    

    RedButtonCentre=(0,-0.003) 
    RedButtonCentre_Calculated = ((RedButtonCentre[0]*np.cos(theta_rad))-(RedButtonCentre[1]*np.sin(theta_rad)),
                                    (RedButtonCentre[1]*np.cos(theta_rad))+(RedButtonCentre[0]*np.sin(theta_rad)))

    RedButtonWorld = (RedButtonCentre_Calculated[0] + RedButtonWorldCoords[0], RedButtonCentre_Calculated[1] + RedButtonWorldCoords[1])

    Task6_Pose=Task1_Pose
    Task6_Pose[0] = RedButtonWorld[0]
    Task6_Pose[1] = RedButtonWorld[1]
    Task6_Pose[2] = 0.250

    tool_rpy = (3.14,0.0,3.14+theta_rad)
    tool_rv = RPYtoRV.rpy2rv(tool_rpy[0],tool_rpy[1],tool_rpy[2]) # Rotational vector values for rotating tool to taskboard orientation
    Task6_Pose[3] = tool_rv[0]
    Task6_Pose[4] = tool_rv[1]
    Task6_Pose[5] = tool_rv[2]

    rtde_c.moveL(Task6_Pose,MaxSpeed,MaxAcceleration)
    time.sleep(SleepTime)

    Task6_Pose[2] = 0.140
    rtde_c.moveL(Task6_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

    Task6_Pose[2] = 0.130
    rtde_c.moveL(Task6_Pose,SlowSpeed,SlowAcceleration)
    time.sleep(SleepTime)

    Task6_Pose[2] = 0.250
    rtde_c.moveL(Task6_Pose,StdSpeed,StdAcceleration)
    time.sleep(SleepTime)

print("Task 6 completed..")


    

In [14]:
## Execution Script

if rtde_c.isConnected() == False:
    print("Robot is disconnected. Trying to reconnect...")
    rtde_c.reconnect()

## Localisation

## Feature detection in Macropose

centre_R_macro = MacroVision.StartMacroPose()

MacroProjectionPoint = ProjectionPlane.GetWorldCoordinates(centre_R_macro,"macro")

MacroWorldCoordinates = [-0.055+MacroProjectionPoint[1],-0.500+MacroProjectionPoint[0]]

print(f"Location of red button in camera coordinate frame : {MacroProjectionPoint}")
print(f"Distance to be moved for orienting the camera coordinate above the red button: {MacroWorldCoordinates}")

rtde_c.moveL([MacroWorldCoordinates[0],MacroWorldCoordinates[1],0.250,0,3.14,0],MaxSpeed,MaxAcceleration)
time.sleep(SleepTime)

## Feature and Orientation in Micropose

centre_R_micro, centre_B_micro = MicroVision.StartMicroPose()

theta = RotationAngle.find_angle(centreR=centre_R_micro, centreB=centre_B_micro)
theta_rad = (theta/180)*np.pi   #converting theta into radians

MicroProjectionPoint = ProjectionPlane.GetWorldCoordinates(centre_R_micro,"micro")
print(f"Location of red button in camera coordinate frame : {MicroProjectionPoint}")

RedButtonWorldCoords = [MacroWorldCoordinates[0] + MicroProjectionPoint[1] + 0.0015, MacroWorldCoordinates[1] - 0.13776 + MicroProjectionPoint[0] -0.0005]
print(f"Distance to be moved for orienting the TCP above red button: {RedButtonWorldCoords}")

## Task Execution order

Task1()             # Blue button pressing
Task2a()            # Yellow slider
Task2B()            # Green slider
Task3()             # Probe replacement
Task4()             # Door Opening
Task4b()            # Probing
Task5()             # Wire stowing
Task6()             # Red button pressing

## Moving to Home position

rtde_c.moveL(Home_Pose,MaxSpeed,MaxAcceleration)
time.sleep(SleepTime)

rtde_c.disconnect()  

Loading Intel Realsense Camera
Pixel coordinates of red  button : (520, 364)
Orientation of Taskboard : 357.70938992500305
Location of red button in camera coordinate frame : (0.06998739391565323, 0.08742465078830719, 0.47)
Distance to be moved for orienting the camera coordinate above the red button: [0.03242465078830719, -0.43001260608434677]
Loading Intel Realsense Camera
centre R: (417, 247)
center B: (466, 251)
355.333141628561
Location of red button in camera coordinate frame : (-0.0036103399470448494, -0.0012223381781950593, 0.172)
Distance to be moved for orienting the TCP above red button: [0.03270231261011213, -0.5718829460313916]
Task1 initiated..
Task1 completed with end pose [0.03410580727320927, -0.5546901364084516, 0.25, 0.13040604024237115, 3.1388177774284065, 0.0024995252288276186]
Task 2 initiated...
Task 3b started..
Loading Intel Realsense Camera
323 404
1057 1142
327 414
716 859
Shape of M5 screen (87, 143, 3)
Centre of Red triangle: (73.5, 58.5)
(73, 58)
centre_R 

: 